In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
import pandas as pd

In [3]:
jokes_df = pd.read_csv("shortjokes.csv")
jokes_df.head(5)

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [43]:
all_jokes = jokes_df["Joke"].tolist()
all_jokes = [joke + '\n' for joke in all_jokes] # add endline char in the end of each joke

In [44]:
vocab = sorted(set(''.join(all_jokes)))
print ('{} unique characters'.format(len(vocab)))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

98 unique characters


In [45]:
all_jokes_as_int = np.array([np.array([char2idx[c] for c in joke]) for joke in all_jokes])
all_jokes_as_int.shape

(231657,)

In [46]:
all_jokes_as_int[1].shape

(119,)

In [60]:
init_dataset = tf.data.Dataset.from_generator(lambda: all_jokes_as_int,
                                         tf.as_dtype(all_jokes_as_int[0].dtype),
                                         tf.TensorShape([None]))

In [61]:
for i in init_dataset.take(5):
    print(''.join(idx2char[i.numpy()]))

[me narrating a documentary about narrators] "I can't hear what they're saying cuz I'm talking"

Telling my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vampires... men.

I've been going through a really rough period at work this week It's my own fault for swapping my tampax for sand paper.

If I could have dinner with anyone, dead or alive... ...I would choose alive. -B.J. Novak-

Two guys walk into a bar. The third guy ducks.



In [62]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

pre_dataset = init_dataset.map(split_input_target)

In [63]:
for input_example, target_example in  pre_dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"'
Target data: 'me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n'


In [64]:
pre_dataset.take(1)

<DatasetV1Adapter shapes: ((?,), (?,)), types: (tf.int32, tf.int32)>

In [65]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 62 ('[')
  expected output: 80 ('m')
Step    1
  input: 80 ('m')
  expected output: 72 ('e')
Step    2
  input: 72 ('e')
  expected output: 3 (' ')
Step    3
  input: 3 (' ')
  expected output: 81 ('n')
Step    4
  input: 81 ('n')
  expected output: 68 ('a')


In [68]:
# Batch size 
BATCH_SIZE = 64
examples_per_epoch = len(all_jokes)
print('examples per epoch:', examples_per_epoch)
steps_per_epoch = examples_per_epoch//BATCH_SIZE
print('steps per epoch:', steps_per_epoch)

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = pre_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, 
                                                    padded_shapes=([None],[None]), 
                                                    padding_values=(tf.constant(char2idx['\n'], dtype=tf.int32),
                                                                    tf.constant(char2idx['\n'], dtype=tf.int32)),
                                                   drop_remainder=True)

# dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset.take(1)

examples per epoch: 231657
steps per epoch: 3619


<DatasetV1Adapter shapes: ((64, ?), (64, ?)), types: (tf.int32, tf.int32)>

In [71]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()[0]])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()[0]])))

Input data:  'What does Mexican binary code consist of? Zeros and Juans.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'
Target data: 'hat does Mexican binary code consist of? Zeros and Juans.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'


In [72]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [73]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [74]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                                           batch_input_shape=[batch_size, None]),
                                 rnn(rnn_units,
                                     return_sequences=True, 
                                     recurrent_initializer='glorot_uniform',
                                     stateful=True),
                                 tf.keras.layers.Dense(vocab_size)
                                ])
    return model

In [75]:
model = build_model(
    vocab_size = len(vocab), 
    embedding_dim=embedding_dim, 
    rnn_units=rnn_units, 
    batch_size=BATCH_SIZE)

In [76]:
for input_example_batch, target_example_batch in dataset.take(1): 
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 200, 98) # (batch_size, sequence_length, vocab_size)


In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           25088     
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 98)            100450    
Total params: 4,063,842
Trainable params: 4,063,842
Non-trainable params: 0
_________________________________________________________________


In [78]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [79]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'Why are blacks excited for 2016? Because it is the year of the monkey.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

Next Char Predictions: 
 'z>~BsI%CC;Q5o,cSU"DaioXoK6Bdpko<\x08_q\'6dd-8`J4)B"1p_3e:dLs)Gyw4!(tZb<E&\x08=\x10.gxytv^"/HZ>%YWi*%4Xe$dl R\\\'s,>+YWd$S/#W"\x08zMu\\}Da6|T"j>Ruq}?ty%:3yt41O_btmJ.My\x103Q`@Q0\'_`\x10:5xKSQPj3BN&szC{Mko[Q\nP5\x10F},G_|-B-aJ=@+'


In [80]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 200, 98)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.584848


In [81]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [82]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [83]:
EPOCHS=3

In [84]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3
3618/3619 [============================>.] - ETA: 0s - loss: 0.9136WARNING:tensorflow:From C:\Users\alisher\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
3619/3619 [==============================] - 209s 58ms/step - loss: 0.9135
Epoch 2/3
3619/3619 [==============================] - 209s 58ms/step - loss: 0.6510
Epoch 3/3
3619/3619 [==============================] - 211s 58ms/step - loss: 0.6079


In [85]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_3'

In [86]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [87]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            25088     
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 98)             100450    
Total params: 4,063,842
Trainable params: 4,063,842
Non-trainable params: 0
_________________________________________________________________


In [88]:
def generate_joke(model, start_string):
    # Evaluation step (generating text using the learned model)
    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
        if idx2char[predicted_id] == '\n':
            break

    return (start_string + ''.join(text_generated))

In [125]:
for _ in range(10):
    print(generate_joke(model, start_string=u"I"))

IN WITEN BRUGHT BARN THY SAVE, ONIFE A BEIGHTWETEH! **ap* * Still training founts. Tell me this to rost 2 people pleased 20K today, but you're taking kodate to apart the political car in the Lord.

IRGORES ORN II MONTHY

Iv just rose a pair of baseball player Mardians and not becoming the strongest. The hill, smoke and whores.

IZ GEREE THE HOOR IN AN AUNGP

IN *TSEPTER**

I] wait to be funny

Ing"...turns out I have a roommate is fun signs.

ICK and Don't Remind Rotaway History.

IRNGENT PUSKMES ON LSKS SATES SITHLEREA' ! . .CrOZe. Your nose. The bolt preserved reposted me off.

ISISIE: guy trollets around his pic. For the man on his driveway, the biggest time-oneth of freedom, with the Virginitis Thoecesson, FUN

